# Analyse : La météo influe-t-elle sur l'humeur ?
## Étude observationnelle - Campus Université de Lille

**Auteurs :** ROUSSEAU Rayane, TCHASSOU Leonel, LOUIS JOSEPH Hugo, MARCOT Solenn  
**Groupe :** 1

Ce notebook analyse les données collectées lors de 29 sessions d'observation.

## 1. Configuration et chargement des données

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuration plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

# Charger les données
df = pd.read_csv('CSVs/meteo_humeur.csv')

# Convertir date et extraire jour de la semaine
df['date'] = pd.to_datetime(df['date'])
df['weekday'] = df['date'].dt.dayofweek
df['weekday_name'] = df['date'].dt.day_name()

# Calculer proportion humeur positive par session
df['mood_pos_prop'] = df['positive_count'] / df['total_evaluated']

print(f"Données chargées : {len(df)} sessions")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../CSVs/meteo_humeur.csv'

### Vérification de la cohérence mathématique

Selon le protocole : `positive_count = facial_pos_count + social_pos_count - both_indicators`

In [ ]:
df['positive_calc'] = df['facial_pos_count'] + df['social_pos_count'] - df['both_indicators']
df['coherence'] = df['positive_count'] == df['positive_calc']

print(f"Cohérence des comptages : {df['coherence'].sum()}/{len(df)} sessions")
print(f"✓ Données valides" if df['coherence'].all() else "✗ ERREUR : incohérences détectées")

## 2. STATISTIQUES DESCRIPTIVES

### 2.1 Vue d'ensemble

In [ ]:
total_individus = df['total_evaluated'].sum()
print(f"Nombre total de sessions : {len(df)}")
print(f"Nombre total d'individus observés : {total_individus}")
print(f"Moyenne individus par session : {df['total_evaluated'].mean():.1f}")

### 2.2 Répartition par catégorie météorologique

In [ ]:
meteo_stats = df.groupby('weather_category').agg({
    'total_evaluated': ['count', 'sum'],
    'positive_count': 'sum',
    'neutral_negative_count': 'sum'
})
meteo_stats.columns = ['nb_sessions', 'nb_individus', 'nb_positifs', 'nb_neutres_neg']
meteo_stats['pct_individus'] = (meteo_stats['nb_individus'] / total_individus * 100).round(1)

print("\nRépartition par catégorie météorologique :")
print(meteo_stats)

# Graphique
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

meteo_stats['nb_sessions'].plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Nombre de sessions par météo')
ax1.set_ylabel('Sessions')
ax1.set_xlabel('')
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

meteo_stats['nb_individus'].plot(kind='bar', ax=ax2, color='lightcoral')
ax2.set_title('Nombre d\'individus observés par météo')
ax2.set_ylabel('Individus')
ax2.set_xlabel('')
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.show()

### 2.3 Répartition par lieu d'observation

In [ ]:
lieu_stats = df.groupby('location').agg({
    'total_evaluated': ['count', 'sum']
})
lieu_stats.columns = ['nb_sessions', 'nb_individus']
lieu_stats['pct'] = (lieu_stats['nb_individus'] / total_individus * 100).round(1)

print("\nRépartition par lieu :")
print(lieu_stats)

# Graphique circulaire
plt.figure(figsize=(8, 6))
plt.pie(lieu_stats['nb_individus'], labels=lieu_stats.index, autopct='%1.1f%%', startangle=90)
plt.title('Répartition des observations par lieu')
plt.show()

### 2.4 Répartition par jour de la semaine

In [ ]:
jour_stats = df.groupby('weekday_name').agg({
    'total_evaluated': ['count', 'sum']
})
jour_stats.columns = ['nb_sessions', 'nb_individus']

print("\nRépartition par jour de la semaine :")
print(jour_stats)

# Événements spéciaux
events = df[df['special_event'].notna() & (df['special_event'] != '')]
print(f"\nSessions avec événements spéciaux : {len(events)}")
if len(events) > 0:
    print(events[['date', 'location', 'weather_category', 'special_event']])

## 3. PROPORTION D'HUMEUR POSITIVE PAR MÉTÉO

### 3.1 Calcul des proportions avec intervalles de confiance à 95%

Formules appliquées :
- Proportion : p = nb_positifs / nb_total
- Erreur standard : SE = √(p(1-p)/n)
- IC 95% : p ± 1.96 × SE

In [ ]:
def calc_proportion_stats(data, category_col, category_val):
    """Calcule proportion, SE et IC 95%"""
    subset = data[data[category_col] == category_val]
    n_total = subset['total_evaluated'].sum()
    n_positive = subset['positive_count'].sum()
    p = n_positive / n_total if n_total > 0 else 0
    se = np.sqrt(p * (1 - p) / n_total) if n_total > 0 else 0
    ic_low = p - 1.96 * se
    ic_high = p + 1.96 * se
    return {
        'n': n_total,
        'n_positive': n_positive,
        'proportion': p,
        'se': se,
        'ic_low': ic_low,
        'ic_high': ic_high
    }

# Calcul pour chaque catégorie météo
meteo_categories = df['weather_category'].unique()
meteo_prop_stats = {}

print("PROPORTIONS D'HUMEUR POSITIVE PAR MÉTÉO\n" + "="*60)
for cat in meteo_categories:
    stats_cat = calc_proportion_stats(df, 'weather_category', cat)
    meteo_prop_stats[cat] = stats_cat
    print(f"\n{cat.upper()} :")
    print(f"  n = {stats_cat['n']} individus")
    print(f"  Proportion humeur+ = {stats_cat['proportion']*100:.1f}%")
    print(f"  Erreur standard = {stats_cat['se']:.4f}")
    print(f"  IC 95% = [{stats_cat['ic_low']*100:.1f}% ; {stats_cat['ic_high']*100:.1f}%]")

### 3.2 Différence entre temps ensoleillé et temps maussade

In [ ]:
if 'sunny' in meteo_prop_stats and 'gloomy' in meteo_prop_stats:
    p1 = meteo_prop_stats['sunny']['proportion']
    p2 = meteo_prop_stats['gloomy']['proportion']
    n1 = meteo_prop_stats['sunny']['n']
    n2 = meteo_prop_stats['gloomy']['n']
    
    diff = p1 - p2
    se_diff = np.sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
    ic_diff_low = diff - 1.96 * se_diff
    ic_diff_high = diff + 1.96 * se_diff
    
    print("\nDIFFÉRENCE SUNNY - GLOOMY :")
    print(f"  Différence observée = {diff*100:.1f} points de pourcentage")
    print(f"  SE de la différence = {se_diff:.4f}")
    print(f"  IC 95% de la différence = [{ic_diff_low*100:.1f} ; {ic_diff_high*100:.1f}]")
    
    if ic_diff_low > 0 or ic_diff_high < 0:
        print("  ✓ L'IC EXCLUT zéro → différence significative au seuil α=0.05")
    else:
        print("  ✗ L'IC INCLUT zéro → différence NON significative au seuil α=0.05")
else:
    print("Catégories sunny et/ou gloomy non présentes dans les données")

### 3.3 Visualisation des proportions avec intervalles de confiance

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Graphique en barres avec IC
categories = list(meteo_prop_stats.keys())
proportions = [meteo_prop_stats[c]['proportion'] * 100 for c in categories]
errors = [1.96 * meteo_prop_stats[c]['se'] * 100 for c in categories]

ax1.bar(categories, proportions, yerr=errors, capsize=8, color='mediumseagreen', alpha=0.7, edgecolor='black')
ax1.set_title('Proportion d\'humeur positive par météo\n(avec IC 95%)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Proportion (%)', fontsize=11)
ax1.set_ylim(0, 100)
ax1.grid(axis='y', alpha=0.3)
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

# Box plot
df.boxplot(column='mood_pos_prop', by='weather_category', ax=ax2)
ax2.set_title('Distribution des proportions d\'humeur positive', fontsize=12, fontweight='bold')
ax2.set_ylabel('Proportion', fontsize=11)
ax2.set_xlabel('Catégorie météorologique', fontsize=11)
plt.suptitle('')

plt.tight_layout()
plt.show()

## 4. ANALYSE DES INDICATEURS COMPORTEMENTAUX

### 4.1 Proportions par indicateur et par météo

In [ ]:
print("ANALYSE DES INDICATEURS COMPORTEMENTAUX\n" + "="*60)

indicateurs_data = []
for cat in meteo_categories:
    subset = df[df['weather_category'] == cat]
    n_total = subset['total_evaluated'].sum()
    n_facial = subset['facial_pos_count'].sum()
    n_social = subset['social_pos_count'].sum()
    n_both = subset['both_indicators'].sum()
    
    p_facial = n_facial / n_total * 100
    p_social = n_social / n_total * 100
    p_both = n_both / n_total * 100
    
    print(f"\n{cat.upper()} :")
    print(f"  Expression faciale positive : {p_facial:.1f}%")
    print(f"  Interaction sociale positive : {p_social:.1f}%")
    print(f"  Les deux indicateurs simultanément : {p_both:.1f}%")
    
    indicateurs_data.append({
        'Météo': cat,
        'Facial': p_facial,
        'Social': p_social,
        'Both': p_both
    })

ind_df = pd.DataFrame(indicateurs_data)

### 4.2 Visualisation des indicateurs

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(ind_df))
width = 0.25

ax.bar(x - width, ind_df['Facial'], width, label='Expression faciale', color='steelblue', alpha=0.8)
ax.bar(x, ind_df['Social'], width, label='Interaction sociale', color='darkorange', alpha=0.8)
ax.bar(x + width, ind_df['Both'], width, label='Les deux', color='mediumseagreen', alpha=0.8)

ax.set_xlabel('Catégorie météorologique', fontsize=11)
ax.set_ylabel('Proportion (%)', fontsize=11)
ax.set_title('Indicateurs comportementaux par catégorie météorologique', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(ind_df['Météo'])
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 5. ANALYSE DES VARIABLES DE CONTRÔLE

### 5.1 Effet du jour de la semaine

In [ ]:
print("EFFET DU JOUR DE LA SEMAINE\n" + "="*60)

jour_prop_data = []
for day in df['weekday_name'].unique():
    stats_day = calc_proportion_stats(df, 'weekday_name', day)
    print(f"{day}: {stats_day['proportion']*100:.1f}% (n={stats_day['n']})")
    jour_prop_data.append({'Jour': day, 'Proportion': stats_day['proportion']*100, 'n': stats_day['n']})

jour_df = pd.DataFrame(jour_prop_data)

# Graphique
plt.figure(figsize=(10, 5))
plt.bar(jour_df['Jour'], jour_df['Proportion'], color='mediumpurple', alpha=0.7, edgecolor='black')
plt.title('Proportion d\'humeur positive par jour de la semaine', fontsize=12, fontweight='bold')
plt.ylabel('Proportion (%)', fontsize=11)
plt.xlabel('Jour', fontsize=11)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 5.2 Effet du lieu d'observation

In [ ]:
print("\nEFFET DU LIEU D'OBSERVATION\n" + "="*60)

lieu_prop_data = []
for lieu in df['location'].unique():
    stats_lieu = calc_proportion_stats(df, 'location', lieu)
    print(f"{lieu}:")
    print(f"  Proportion = {stats_lieu['proportion']*100:.1f}%")
    print(f"  n = {stats_lieu['n']}")
    print(f"  IC 95% = [{stats_lieu['ic_low']*100:.1f}% ; {stats_lieu['ic_high']*100:.1f}%]")
    lieu_prop_data.append({'Lieu': lieu, 'Proportion': stats_lieu['proportion']*100})

# Graphique
lieu_df = pd.DataFrame(lieu_prop_data)
plt.figure(figsize=(8, 5))
plt.bar(lieu_df['Lieu'], lieu_df['Proportion'], color='orange', alpha=0.7, edgecolor='black')
plt.title('Proportion d\'humeur positive par lieu', fontsize=12, fontweight='bold')
plt.ylabel('Proportion (%)', fontsize=11)
plt.xlabel('Lieu', fontsize=11)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 5.3 Effet des événements spéciaux

In [ ]:
df['has_event'] = df['special_event'].notna() & (df['special_event'] != '')

if df['has_event'].sum() > 0:
    print("\nEFFET DES ÉVÉNEMENTS SPÉCIAUX\n" + "="*60)
    for has_ev in [True, False]:
        subset = df[df['has_event'] == has_ev]
        n_tot = subset['total_evaluated'].sum()
        n_pos = subset['positive_count'].sum()
        p = n_pos / n_tot * 100 if n_tot > 0 else 0
        label = "Avec événement spécial" if has_ev else "Sans événement spécial"
        print(f"{label}: {p:.1f}% (n={n_tot})")
else:
    print("\nAucun événement spécial noté durant les observations")

## 6. PARAMÈTRES MÉTÉOROLOGIQUES DÉTAILLÉS

### 6.1 Corrélations avec la proportion d'humeur positive

In [ ]:
meteo_vars = ['precip_mm', 'humidity_pct', 'app_temp_C', 'visibility_m', 
              'wind_speed_kmh', 'gusts_kmh', 'cloud_pct']

print("CORRÉLATIONS PARAMÈTRES MÉTÉO - HUMEUR\n" + "="*60)

correlations = {}
for var in meteo_vars:
    if var in df.columns:
        corr = df[var].corr(df['mood_pos_prop'])
        correlations[var] = corr
        print(f"{var:20s}: r = {corr:6.3f}")

# Identifier les corrélations les plus fortes
corr_sorted = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)
print(f"\nParamètre le plus corrélé: {corr_sorted[0][0]} (r={corr_sorted[0][1]:.3f})")

### 6.2 Visualisation des corrélations

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, var in enumerate(['app_temp_C', 'cloud_pct', 'precip_mm', 'humidity_pct', 'wind_speed_kmh', 'visibility_m']):
    if idx < len(axes) and var in df.columns:
        ax = axes[idx]
        ax.scatter(df[var], df['mood_pos_prop'] * 100, alpha=0.6, s=50)
        
        # Droite de régression
        z = np.polyfit(df[var].dropna(), df.loc[df[var].notna(), 'mood_pos_prop'] * 100, 1)
        p = np.poly1d(z)
        x_line = np.linspace(df[var].min(), df[var].max(), 100)
        ax.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2)
        
        ax.set_xlabel(var.replace('_', ' ').title(), fontsize=10)
        ax.set_ylabel('Humeur+ (%)', fontsize=10)
        ax.set_title(f'r = {correlations.get(var, 0):.3f}', fontsize=11)
        ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 6.3 Matrice de corrélations

In [ ]:
corr_matrix = df[meteo_vars + ['mood_pos_prop']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={'shrink': 0.8})
plt.title('Matrice de corrélations - Paramètres météo et humeur', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

### 6.4 Régression linéaire multiple

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X = df[meteo_vars].fillna(df[meteo_vars].mean())
y = df['mood_pos_prop']

model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)

print("RÉGRESSION LINÉAIRE MULTIPLE\n" + "="*60)
print(f"R² = {r2:.3f} ({r2*100:.1f}% de variance expliquée)")
print(f"\nCoefficients des prédicteurs:")

coef_df = pd.DataFrame({
    'Variable': meteo_vars,
    'Coefficient': model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(coef_df.to_string(index=False))
print(f"\nIntercept: {model.intercept_:.4f}")

## 7. TESTS STATISTIQUES DE SIGNIFICATIVITÉ

### 7.1 Test Z de comparaison de deux proportions (Sunny vs Gloomy)

**Hypothèses:**
- H0 : p_sunny = p_gloomy (pas de différence)
- H1 : p_sunny ≠ p_gloomy (différence significative)
- Seuil α = 0.05 (défini a priori)

**Formules:**
- p_pooled = (x1 + x2) / (n1 + n2)
- SE = √(p_pooled × (1-p_pooled) × (1/n1 + 1/n2))
- Z = (p1 - p2) / SE
- p-valeur = 2 × P(Z > |z|)

In [ ]:
def test_two_proportions(n1, p1, n2, p2):
    """Test Z de comparaison de deux proportions"""
    x1 = int(n1 * p1)
    x2 = int(n2 * p2)
    
    # Proportion pooled
    p_pooled = (x1 + x2) / (n1 + n2)
    
    # Erreur standard
    se = np.sqrt(p_pooled * (1 - p_pooled) * (1/n1 + 1/n2))
    
    # Z-statistic
    z = (p1 - p2) / se if se > 0 else 0
    
    # p-valeur (test bilatéral)
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    
    # Cohen's h (taille d'effet)
    h = 2 * (np.arcsin(np.sqrt(p1)) - np.arcsin(np.sqrt(p2)))
    
    return {
        'z': z,
        'p_value': p_value,
        'cohen_h': h,
        'p_pooled': p_pooled,
        'se': se
    }

# Test sunny vs gloomy
if 'sunny' in meteo_prop_stats and 'gloomy' in meteo_prop_stats:
    n1 = meteo_prop_stats['sunny']['n']
    p1 = meteo_prop_stats['sunny']['proportion']
    n2 = meteo_prop_stats['gloomy']['n']
    p2 = meteo_prop_stats['gloomy']['proportion']
    
    test_result = test_two_proportions(n1, p1, n2, p2)
    
    print("TEST Z : SUNNY vs GLOOMY\n" + "="*60)
    print(f"Sunny  : n={n1}, p={p1*100:.1f}%")
    print(f"Gloomy : n={n2}, p={p2*100:.1f}%")
    print(f"\nDifférence : {(p1-p2)*100:.1f} points de pourcentage")
    print(f"\nRésultats du test:")
    print(f"  p_pooled = {test_result['p_pooled']:.4f}")
    print(f"  SE = {test_result['se']:.4f}")
    print(f"  Z-statistic = {test_result['z']:.3f}")
    print(f"  p-valeur = {test_result['p_value']:.4f}")
    print(f"\n  {'✓ SIGNIFICATIF' if test_result['p_value'] < 0.05 else '✗ NON SIGNIFICATIF'} au seuil α=0.05")
    
    # Interprétation Cohen's h
    h = abs(test_result['cohen_h'])
    if h < 0.2:
        effect_size = "PETIT"
    elif h < 0.5:
        effect_size = "MOYEN"
    else:
        effect_size = "GRAND"
    
    print(f"\nTaille d'effet (Cohen's h):")
    print(f"  h = {test_result['cohen_h']:.3f}")
    print(f"  Interprétation: effet {effect_size}")
    print(f"  (h<0.2=petit, 0.2≤h<0.5=moyen, h≥0.5=grand)")
else:
    print("Catégories sunny et/ou gloomy non disponibles")

### 7.2 Puissance statistique a posteriori

In [ ]:
if 'sunny' in meteo_prop_stats and 'gloomy' in meteo_prop_stats:
    try:
        from statsmodels.stats.power import zt_ind_solve_power
        
        power = zt_ind_solve_power(
            effect_size=abs(test_result['cohen_h']),
            nobs1=n1,
            alpha=0.05,
            ratio=n2/n1,
            alternative='two-sided'
        )
        
        print("PUISSANCE STATISTIQUE A POSTERIORI\n" + "="*60)
        print(f"Puissance observée = {power:.3f} ({power*100:.1f}%)")
        print(f"Objectif fixé a priori = 0.80 (80%)")
        print(f"\n{'✓ Puissance SUFFISANTE' if power >= 0.80 else '✗ Puissance INSUFFISANTE'}")
        
        if power < 0.80:
            print(f"\nNote: Une taille d'échantillon plus importante aurait été nécessaire")
            print(f"pour atteindre la puissance de 80% visée.")
    except Exception as e:
        print(f"Calcul de puissance non disponible: {e}")

### 7.3 Tests complémentaires (si catégorie 'special' disponible)

In [ ]:
if 'special' in meteo_prop_stats:
    print("TESTS COMPLÉMENTAIRES\n" + "="*60)
    
    # Sunny vs Special
    if 'sunny' in meteo_prop_stats:
        n1 = meteo_prop_stats['sunny']['n']
        p1 = meteo_prop_stats['sunny']['proportion']
        n3 = meteo_prop_stats['special']['n']
        p3 = meteo_prop_stats['special']['proportion']
        
        test_ss = test_two_proportions(n1, p1, n3, p3)
        print(f"\nSunny vs Special:")
        print(f"  Z = {test_ss['z']:.3f}, p = {test_ss['p_value']:.4f}")
        print(f"  {'Significatif' if test_ss['p_value'] < 0.05 else 'Non significatif'}")
    
    # Gloomy vs Special
    if 'gloomy' in meteo_prop_stats:
        n2 = meteo_prop_stats['gloomy']['n']
        p2 = meteo_prop_stats['gloomy']['proportion']
        
        test_gs = test_two_proportions(n2, p2, n3, p3)
        print(f"\nGloomy vs Special:")
        print(f"  Z = {test_gs['z']:.3f}, p = {test_gs['p_value']:.4f}")
        print(f"  {'Significatif' if test_gs['p_value'] < 0.05 else 'Non significatif'}")
    
    print(f"\nNote: n={n3} pour la catégorie 'special', interprétation limitée par la taille d'échantillon.")
else:
    print("Pas de catégorie 'special' dans les données")

## 8. SYNTHÈSE DES RÉSULTATS POUR LE RAPPORT

### 8.1 Tableau récapitulatif final

In [ ]:
print("\n" + "="*80)
print("SYNTHÈSE FINALE POUR LE RAPPORT")
print("="*80)

summary_data = []
for cat in meteo_categories:
    stats = meteo_prop_stats[cat]
    summary_data.append({
        'Météo': cat.upper(),
        'N individus': stats['n'],
        'N positifs': stats['n_positive'],
        'Proportion (%)': f"{stats['proportion']*100:.1f}",
        'IC 95%': f"[{stats['ic_low']*100:.1f} ; {stats['ic_high']*100:.1f}]",
        'SE': f"{stats['se']:.4f}"
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + summary_df.to_string(index=False))

if 'sunny' in meteo_prop_stats and 'gloomy' in meteo_prop_stats:
    print(f"\n\nRÉSULTAT DU TEST STATISTIQUE PRINCIPAL:")
    print(f"  Différence sunny-gloomy: {(p1-p2)*100:.1f} points")
    print(f"  Z-statistic: {test_result['z']:.3f}")
    print(f"  p-valeur: {test_result['p_value']:.4f}")
    print(f"  Cohen's h: {test_result['cohen_h']:.3f} (effet {effect_size})")
    print(f"  Conclusion: {'REJET de H0' if test_result['p_value'] < 0.05 else 'NON-REJET de H0'} au seuil α=0.05")

### 8.2 Points clés à inclure dans le rapport

In [ ]:
print("\n" + "="*80)
print("POINTS CLÉS POUR LA RÉDACTION DU RAPPORT")
print("="*80)

print(f"\n1. STATISTIQUES DESCRIPTIVES:")
print(f"   - {len(df)} sessions d'observation réalisées")
print(f"   - {total_individus} individus évalués au total")
print(f"   - Répartition météo: {meteo_stats.to_dict()}")

print(f"\n2. RÉSULTAT PRINCIPAL:")
if 'sunny' in meteo_prop_stats and 'gloomy' in meteo_prop_stats:
    print(f"   - Sunny: {p1*100:.1f}%, IC=[{meteo_prop_stats['sunny']['ic_low']*100:.1f}; {meteo_prop_stats['sunny']['ic_high']*100:.1f}]")
    print(f"   - Gloomy: {p2*100:.1f}%, IC=[{meteo_prop_stats['gloomy']['ic_low']*100:.1f}; {meteo_prop_stats['gloomy']['ic_high']*100:.1f}]")
    print(f"   - Différence: {(p1-p2)*100:.1f} points")
    print(f"   - Significativité: p={test_result['p_value']:.4f} ({'<' if test_result['p_value'] < 0.05 else '>'} 0.05)")

print(f"\n3. TAILLE D'EFFET:")
if 'sunny' in meteo_prop_stats and 'gloomy' in meteo_prop_stats:
    print(f"   - Cohen's h = {test_result['cohen_h']:.3f}")
    print(f"   - Interprétation: effet {effect_size}")

print(f"\n4. CORRÉLATIONS PARAMÈTRES MÉTÉO:")
for var, corr in sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)[:3]:
    print(f"   - {var}: r={corr:.3f}")

print(f"\n5. RÉGRESSION MULTIPLE:")
print(f"   - R² = {r2:.3f} ({r2*100:.1f}% de variance expliquée)")

print(f"\n6. PUISSANCE STATISTIQUE:")
if 'power' in locals():
    print(f"   - Observée: {power:.3f} ({'≥' if power >= 0.80 else '<'} 0.80 visé)")

print("\n" + "="*80)

## 9. EXPORT DES RÉSULTATS

In [ ]:
# Sauvegarder les statistiques principales
results_export = {
    'statistiques_descriptives': meteo_stats.to_dict(),
    'proportions_par_meteo': {k: v for k, v in meteo_prop_stats.items()},
    'correlations': correlations,
    'r_squared': r2
}

if 'test_result' in locals():
    results_export['test_statistique'] = test_result

# Afficher pour copier-coller dans le rapport
import json
print("\nRésultats exportables (JSON):")
print(json.dumps(results_export, indent=2, default=str))

print("\n✓ Analyse terminée ! Utilisez ces résultats pour compléter le rapport.")